In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import sys
import os
import re
import pysam
from Bio.Seq import Seq
import pyranges as pr
from rapidfuzz import fuzz
import mappy as mp
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dpath = "/nfs/turbo/umms-indikar/shared/projects/HSC/data/results/HSC/demultiplex/"
df = []

nrows = 100000

for f in os.listdir(dpath):
    if 'putative_bc' in f:
        sample_name = f.split(".")[0]
        fpath = f"{dpath}{f}"
        tmp = pd.read_csv(fpath, nrows=nrows)
        tmp['sample_name'] = sample_name
        df.append(tmp)
    
df = pd.concat(df)
df.head()

,read_id,putative_bc,putative_bc_min_q,putative_umi,umi_end,pre_bc_flanking,post_umi_flanking,sample_name
0,dcf36249-ec3f-46eb-ba52-1c25fc42262e,CTCTCTCTCTCTTTTT,8.0,TTTTTTTTTTTT,-41.0,CCTCT,CTTTT,P2r5
1,de97d394-99df-4058-ba80-6b739ec8637f,NaN,NaN,NaN,NaN,NaN,NaN,P2r5
2,3eb0e4f7-8e41-4d2b-9bdc-dfde8be57180,NaN,NaN,NaN,NaN,NaN,NaN,P2r5
3,91ba75ae-f088-40f8-b252-406a10bb697e,NaN,NaN,NaN,NaN,NaN,NaN,P2r5
4,0add03ab-eeae-4ad7-80f9-71314043d2f8,NaN,NaN,NaN,NaN,NaN,NaN,P2r5


In [4]:
pdf = df.copy()

pdf['strand'] = np.where(pdf['umi_end'].isna(), 'NA', 'positive')
pdf['strand'] = np.where(pdf['umi_end'] < 0, 'negative', pdf['strand'])

# group reads by strand
gx = pdf.groupby(['sample_name', 'strand'])['read_id'].nunique()
gx = gx.reset_index()
gx['sample_totals'] = gx.groupby(['sample_name'])['read_id'].transform('sum')
gx['percent'] = gx['read_id'] / gx['sample_totals']
gx = pd.pivot_table(gx, 
                    index='sample_name',
                    columns='strand',
                    values='percent')

gx

strand,NA,negative,positive
sample_name,,,
Gridr1,0.11442,0.41510,0.47048
Gridr2,0.12278,0.40811,0.46911
Gridr3,0.13071,0.41205,0.45724
Gridr4,0.13552,0.40382,0.46066
P2r5,0.09990,0.76927,0.13083
P2r6,0.09985,0.75637,0.14378
P2r7,0.10657,0.73909,0.15434


In [ ]:
fpath = "/nfs/turbo/umms-indikar/shared/projects/HSC/data/results/HSC/demultiplex/P2r7.putative_bc.csv"

df = pd.read_csv(fpath, nrows=100000)
df['direction'] = np.sign(df['umi_end'])

df['direction'].value_counts(dropna=False, normalize=True)